# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 7 2022 2:56PM,250215,10,0086145259,ISDIN Corporation,Released
1,Nov 7 2022 2:56PM,250215,10,0086145266,ISDIN Corporation,Released
2,Nov 7 2022 2:56PM,250215,10,0086145270,ISDIN Corporation,Released
3,Nov 7 2022 2:56PM,250215,10,0086145279,ISDIN Corporation,Released
4,Nov 7 2022 2:56PM,250215,10,0086145280,ISDIN Corporation,Released
5,Nov 7 2022 2:56PM,250215,10,0086145281,ISDIN Corporation,Released
6,Nov 7 2022 2:56PM,250215,10,0086145295,ISDIN Corporation,Released
7,Nov 7 2022 2:56PM,250215,10,0086145296,ISDIN Corporation,Released
8,Nov 7 2022 2:56PM,250215,10,0086146028,ISDIN Corporation,Released
9,Nov 7 2022 2:56PM,250215,10,0086146027,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
49,250212,Released,20
50,250213,Released,63
51,250214,Released,1
52,250215,Released,38
53,250216,Released,7


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
250212,NaN,NaN,20.0
250213,NaN,NaN,63.0
250214,NaN,NaN,1.0
250215,NaN,NaN,38.0
250216,NaN,NaN,7.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
250118,5.0,1.0,0.0
250121,0.0,1.0,0.0
250130,0.0,0.0,1.0
250131,0.0,0.0,1.0
250133,1.0,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
250118,5,1,0
250121,0,1,0
250130,0,0,1
250131,0,0,1
250133,1,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,250118,5,1,0
1,250121,0,1,0
2,250130,0,0,1
3,250131,0,0,1
4,250133,1,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,250118,5,1,
1,250121,,1,
2,250130,,,1
3,250131,,,1
4,250133,1,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 7 2022 2:56PM,250215,10,ISDIN Corporation
38,Nov 7 2022 2:56PM,250213,10,ISDIN Corporation
101,Nov 7 2022 2:56PM,250205,10,ISDIN Corporation
117,Nov 7 2022 2:56PM,250203,10,ISDIN Corporation
172,Nov 7 2022 2:55PM,250216,10,"Snap Medical Industries, LLC"
179,Nov 7 2022 2:50PM,250212,10,Eywa Pharma Inc.
199,Nov 7 2022 2:49PM,250211,10,Eywa Pharma Inc.
204,Nov 7 2022 2:46PM,250214,19,Methapharm-G
205,Nov 7 2022 2:29PM,250207,16,Sartorius Bioprocess Solutions
207,Nov 7 2022 2:27PM,250206,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Nov 7 2022 2:56PM,250215,10,ISDIN Corporation,,,38
1,Nov 7 2022 2:56PM,250213,10,ISDIN Corporation,,,63
2,Nov 7 2022 2:56PM,250205,10,ISDIN Corporation,,,16
3,Nov 7 2022 2:56PM,250203,10,ISDIN Corporation,,,55
4,Nov 7 2022 2:55PM,250216,10,"Snap Medical Industries, LLC",,,7
5,Nov 7 2022 2:50PM,250212,10,Eywa Pharma Inc.,,,20
6,Nov 7 2022 2:49PM,250211,10,Eywa Pharma Inc.,,,5
7,Nov 7 2022 2:46PM,250214,19,Methapharm-G,,,1
8,Nov 7 2022 2:29PM,250207,16,Sartorius Bioprocess Solutions,,,2
9,Nov 7 2022 2:27PM,250206,21,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 7 2022 2:56PM,250215,10,ISDIN Corporation,38,,
1,Nov 7 2022 2:56PM,250213,10,ISDIN Corporation,63,,
2,Nov 7 2022 2:56PM,250205,10,ISDIN Corporation,16,,
3,Nov 7 2022 2:56PM,250203,10,ISDIN Corporation,55,,
4,Nov 7 2022 2:55PM,250216,10,"Snap Medical Industries, LLC",7,,
5,Nov 7 2022 2:50PM,250212,10,Eywa Pharma Inc.,20,,
6,Nov 7 2022 2:49PM,250211,10,Eywa Pharma Inc.,5,,
7,Nov 7 2022 2:46PM,250214,19,Methapharm-G,1,,
8,Nov 7 2022 2:29PM,250207,16,Sartorius Bioprocess Solutions,2,,
9,Nov 7 2022 2:27PM,250206,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 7 2022 2:56PM,250215,10,ISDIN Corporation,38,,
1,Nov 7 2022 2:56PM,250213,10,ISDIN Corporation,63,,
2,Nov 7 2022 2:56PM,250205,10,ISDIN Corporation,16,,
3,Nov 7 2022 2:56PM,250203,10,ISDIN Corporation,55,,
4,Nov 7 2022 2:55PM,250216,10,"Snap Medical Industries, LLC",7,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 7 2022 2:56PM,250215,10,ISDIN Corporation,38.0,NaN,NaN
1,Nov 7 2022 2:56PM,250213,10,ISDIN Corporation,63.0,NaN,NaN
2,Nov 7 2022 2:56PM,250205,10,ISDIN Corporation,16.0,NaN,NaN
3,Nov 7 2022 2:56PM,250203,10,ISDIN Corporation,55.0,NaN,NaN
4,Nov 7 2022 2:55PM,250216,10,"Snap Medical Industries, LLC",7.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 7 2022 2:56PM,250215,10,ISDIN Corporation,38.0,0.0,0.0
1,Nov 7 2022 2:56PM,250213,10,ISDIN Corporation,63.0,0.0,0.0
2,Nov 7 2022 2:56PM,250205,10,ISDIN Corporation,16.0,0.0,0.0
3,Nov 7 2022 2:56PM,250203,10,ISDIN Corporation,55.0,0.0,0.0
4,Nov 7 2022 2:55PM,250216,10,"Snap Medical Industries, LLC",7.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3502660,219.0,19.0,16.0
15,1250792,121.0,22.0,37.0
16,1000654,4.0,1.0,0.0
18,250131,1.0,0.0,0.0
19,2001432,17.0,6.0,5.0
20,250184,1.0,0.0,0.0
21,1501001,6.0,0.0,0.0
22,1000772,4.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3502660,219.0,19.0,16.0
1,15,1250792,121.0,22.0,37.0
2,16,1000654,4.0,1.0,0.0
3,18,250131,1.0,0.0,0.0
4,19,2001432,17.0,6.0,5.0
5,20,250184,1.0,0.0,0.0
6,21,1501001,6.0,0.0,0.0
7,22,1000772,4.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,219.0,19.0,16.0
1,15,121.0,22.0,37.0
2,16,4.0,1.0,0.0
3,18,1.0,0.0,0.0
4,19,17.0,6.0,5.0
5,20,1.0,0.0,0.0
6,21,6.0,0.0,0.0
7,22,4.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,219.0
1,15,Released,121.0
2,16,Released,4.0
3,18,Released,1.0
4,19,Released,17.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,21,22
Status,,,,,,,,
Completed,16.0,37.0,0.0,0.0,5.0,0.0,0.0,0.0
Executing,19.0,22.0,1.0,0.0,6.0,0.0,0.0,0.0
Released,219.0,121.0,4.0,1.0,17.0,1.0,6.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,21,22
0,Completed,16.0,37.0,0.0,0.0,5.0,0.0,0.0,0.0
1,Executing,19.0,22.0,1.0,0.0,6.0,0.0,0.0,0.0
2,Released,219.0,121.0,4.0,1.0,17.0,1.0,6.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,21,22
0,Completed,16.0,37.0,0.0,0.0,5.0,0.0,0.0,0.0
1,Executing,19.0,22.0,1.0,0.0,6.0,0.0,0.0,0.0
2,Released,219.0,121.0,4.0,1.0,17.0,1.0,6.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()